In [81]:
from argparse import Namespace
from transformers import (
    BertForMaskedLM,
    BertTokenizer,
)
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
token_path = 'georgian-vocab.txt'

In [13]:
tok = BertTokenizer(
    args.token_path
)

In [99]:
from transformers import pipeline
fill_mask = pipeline(
    "fill-mask",
    model='model/checkpoint-30000',
    tokenizer=tok
)

In [71]:
def generate_sentence(sent,n):
    for i in range(n):
        sent+=" [MASK]"
        sent = fill_mask(sent)[0]['sequence']
    return sent

In [103]:
generate_sentence('თიბისი ბანკის ',6)

'თიბისი ბანკის თანამშრომელია საქართველოში საქართველოში საქართველოში საქართველოში საქართველოში'

In [80]:
generate_sentence('საქართველოში მიიღეს მონაწილეობა თიბისი',6)

'საქართველოში მიიღეს მონაწილეობა თიბისი ბანკის მიერ მონაწილეობა მიიღო მონაწილეობა მიიღო'

In [84]:
def generate_beam(sent,n,beam):
    answer = fill_mask(sent+" [MASK]")
    words = [word['sequence'] for word in answer]
    all_scores = [-np.log(o['score']) for o in answer] 
    top_k_words = words[:beam]
    top_k_scores = all_scores[:beam]
    scores = all_scores
    for j in range(n):
        all_scores = []
        all_sequences = []
        for i in range(beam):    
            out = fill_mask(top_k_words[i]+" [MASK]")
            scores = [-np.log(o['score']) for o in out]
            all_scores.extend(scores)
        
        words = [o[]]
        print(scores)

In [85]:
generate_beam('გამარჯობა ქვეყანა',1,0)

[1.7865485437849875, 3.1716956740842575, 3.9279808596597876, 4.359411155078125, 4.367350947353083]


In [51]:
fill_mask('საქართველოს [MASK] მიიღეს კანონი')

[{'score': 0.02661210112273693,
  'token': 417,
  'token_str': 'შ ე ს ა ხ ე ბ',
  'sequence': 'საქართველოს შესახებ მიიღეს კანონი'},
 {'score': 0.017453715205192566,
  'token': 437,
  'token_str': 'მ ი ე რ',
  'sequence': 'საქართველოს მიერ მიიღეს კანონი'},
 {'score': 0.017082579433918,
  'token': 125,
  'token_str': 'ა რ',
  'sequence': 'საქართველოს არ მიიღეს კანონი'},
 {'score': 0.016533562913537025,
  'token': 3710,
  'token_str': 'მ ა ს შ ტ ა ბ ი თ',
  'sequence': 'საქართველოს მასშტაბით მიიღეს კანონი'},
 {'score': 0.015364976599812508,
  'token': 12070,
  'token_str': 'ბ ი ზ ნ ე ს ო მ ბ უ დ ს მ ე ნ ი',
  'sequence': 'საქართველოს ბიზნესომბუდსმენი მიიღეს კანონი'}]

In [56]:
fill_mask('საქართველოში ბენზინის ფასი ძალიან [MASK]')

[{'score': 0.05805490165948868,
  'token': 283,
  'token_str': 'ა რ ი ს',
  'sequence': 'საქართველოში ბენზინის ფასი ძალიან არის'},
 {'score': 0.017442883923649788,
  'token': 557,
  'token_str': 'დ ი დ ი',
  'sequence': 'საქართველოში ბენზინის ფასი ძალიან დიდი'},
 {'score': 0.01632664166390896,
  'token': 2446,
  'token_str': 'მ ნ ი შ ვ ნ ე ლ ო ვ ა ნ ი ა',
  'sequence': 'საქართველოში ბენზინის ფასი ძალიან მნიშვნელოვანია'},
 {'score': 0.01518868375569582,
  'token': 358,
  'token_str': 'ი ყ ო',
  'sequence': 'საქართველოში ბენზინის ფასი ძალიან იყო'},
 {'score': 0.014982525259256363,
  'token': 1061,
  'token_str': 'ა რ ი ა ნ',
  'sequence': 'საქართველოში ბენზინის ფასი ძალიან არიან'}]

In [40]:
import math
def continue_sent(sent):
    sent = sent + '[MASK]'
    predictions = fill_mask(sent)
    return predictions[0], predictions[1]

In [41]:
def continue_with_beam(sent, topk=5, n=2):
    output_sequences = [(sent, 0)]
    for _ in range (n):
        new_sequences = []
        for sentence_longer, score in output_sequences:
            for predictions in continue_sent(sentence_longer):
                new_sent, new_score = predictions['sequence'], predictions['score']
                new_sent = new_sent.rstrip('.</s>').lstrip('<s>')
                new_sequences += [(new_sent, score+math.log(new_score))]
        if (len(new_sequences) == 0): return output_sequences
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:topk]
    
    return output_sequences

In [42]:
continue_with_beam('საქართველოსში მიიღეს კანონი ',10)

PipelineException: No mask_token ([MASK]) found on the input